In [1]:
from sql_engine import connect
import datetime
import pandas as pd

In [11]:
diff_days_start = 2
diff_days_end = 2
balance_apply_SQL = f'''
	
SELECT
		CAST(
		t1.id AS CHAR ( 30 )) id,
		t1.trans_time,
		LEFT ( t1.trans_time, 10 ) trans_date,
		t1.trans_no SourceNo,
		CAST(
		t3.user_id AS CHAR ( 30 )) user_id,
		t3.union_id UnionNumber,
		t3.name name,
		t2.out_id activityId,
        t2.policy_name,
		t1.title,
		t1.body,
	IF
		( t1.amt_trans > 0, t1.amt_trans / 100, 0 ) amt_in,
	IF
		( t1.amt_trans < 0, t1.amt_trans / 100, 0 ) amt_out,
	IF
		( t1.amt_trans > 0, t1.amt_trans / 100, 0 ) +
	IF
		( t1.amt_trans < 0, t1.amt_trans / 100, 0 ) money
	FROM
		oryx_account.account_trans_record t1
		INNER JOIN oryx_policy.policy t2 ON t1.policy_id = t2.policy_id
		INNER JOIN oryx_agent.agent_info t3 ON t1.customer_id = t3.user_id
		left join oryx_bi.zclx t6 on t2.out_id = t6.zc_id
		
	WHERE
		t1.company_id IN ( 12, 979 ) 
		AND t6.ActivityCWTypeNo in (1,2)
		and t2.state = 1
		AND t1.customer_type = 0 
		AND t1.account_type = 0 
		AND ( t1.type NOT IN ( 'pay', 'refund' ) OR t1.title = '余额申请单' ) 
		AND t1.title NOT IN ( '历史订单取消发货单余额退款', '车票审核补报上账', '历史订单余额退款', '车票审核报销上账' ) 
		AND DATEDIFF( NOW(), t1.trans_time )<={diff_days_start} AND DATEDIFF(NOW(), t1.trans_time)>={diff_days_end}
		AND t1.policy_id IS NOT NULL 
		AND ( t1.remark NOT LIKE '%%测试%%' OR t1.remark IS NULL ) 
		and title = '余额申请单'
		AND t3.union_id not in ('CM66381516','CM00006317','CM00007650')
        '''

In [12]:
yx_con, server = connect('yx')

In [13]:
balance_apply_data = pd.read_sql(balance_apply_SQL, yx_con)

2021-04-24 03:00:39,968| ERROR   | Socket exception: 远程主机强迫关闭了一个现有的连接。 (10054)
2021-04-24 03:00:40,958| ERROR   | Socket exception: 远程主机强迫关闭了一个现有的连接。 (10054)
2021-04-24 03:00:40,959| ERROR   | Socket exception: 远程主机强迫关闭了一个现有的连接。 (10054)
